In [1]:
import json
import yaml
import sys
import time
import IPython.display as ipd
import pprint
from pathlib import Path
from tqdm import tqdm

import numpy as np
import torch
import torchaudio
from librosa.filters import mel as librosa_mel_fn
#import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy.io.wavfile import write


import toybox

In [2]:
#run_tfkfulmask_k3,run_tfkfulplus_k3 by gradtfkfultts
model_name = 'gradfreqktts' #  gradseptts, gradtfktts, gradtfk5tts, gradtimektts, gradfreqktts, gradtfkful_mask, gradtfkful_plus
runtime_name = 'infer4colb'
hifigan_versions = 'LJ_V1'
wav_dir_name = f'wav_{hifigan_versions}'
test_ds_path = Path('configs/test_dataset.json') #Path(config['test_datalist_path'])

config = toybox.load_yaml_and_expand_var('configs/config_exp_mid.yaml')
print(model_name)
print(runtime_name)
n_mels: int = config['n_mels'] # 80
n_fft: int = config['n_fft'] # 1024
sample_rate: int = config['sample_rate'] # 22050
hop_size: int = config['hop_size'] # 256
win_size: int = config['win_size'] # 1024
f_min: int = config['f_min'] # 0
f_max: int = config['f_max'] # 8000
random_seed: int = 1234#config['random_seed'] # 42 or 1234
print(n_mels, n_fft, sample_rate, hop_size, win_size, f_min, f_max, random_seed)

toybox.set_seed(random_seed)

gradfreqktts
infer4colb
80 1024 22050 256 1024 0 8000 1234


In [3]:
#devicesetting
import os

print(f"all cpu at using device: {os.cpu_count()}")
print(f"Number of available CPU: {len(os.sched_getaffinity(0))}") # Number of available CPUs can also be obtained. ,use systemcall at linux.
print(f"GPU_name: {torch.cuda.get_device_name()}\nGPU avail: {torch.cuda.is_available()}\n")

all cpu at using device: 52
Number of available CPU: 4


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [4]:
DEVICE = 'cpu' #'cuda' or 'cpu'
#DEVICE_HiFiGAN = 'cuda'
#if torch.cuda.is_available():
#    print('use cuda')
#else:
#    os._exit(os.EX_OK)
#    print('use cpu')

#device = torch.device(DEVICE_HiFiGAN)
device = torch.device(DEVICE)
print(f'device: {device}')

device: cpu


In [5]:
RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/e500_n50')
IMEL_DIR_PATH = RESULT_DIR_PATH / 'mel'
RESULT_WAV_DIR_PATH = RESULT_DIR_PATH / wav_dir_name
#RESULT_JSON_PATH = RESULT_DIR_PATH / f'eval4mid_{hifigan_versions}.json'
#eval_jsonl_path = RESULT_DIR_PATH / 'eval4mid.json'
RESULT_JSON_PATH = RESULT_DIR_PATH / f'eval4mid_{hifigan_versions}.json'

print(RESULT_DIR_PATH)
print(IMEL_DIR_PATH)
print(RESULT_WAV_DIR_PATH)

result4eval/infer4colb/gradfreqktts/cpu/e500_n50
result4eval/infer4colb/gradfreqktts/cpu/e500_n50/mel
result4eval/infer4colb/gradfreqktts/cpu/e500_n50/wav_LJ_V1


In [6]:
# setting path
#IMEL_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/mel')

"""
if model_name == 'groundtruth':
    IMEL_DIR_PATH = Path('./result4eval/infer4PBL/groundtruth/cuda/mel/')
else:
    IMEL_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/mel')
"""

#RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}')
#RESULT_WAV_DIR_PATH = RESULT_DIR_PATH / wav_dir_name

print('-------------------------------------------')
if test_ds_path.exists():
    print(f'Exists {str(test_ds_path)}')
    with open(config['test_datalist_path']) as j:
        test_ds_list = json.load(j)
    print(f'loaded {test_ds_path}')
else:
    print(f'No exist {test_ds_path}')

print('-------------------------------------------')
if RESULT_DIR_PATH.exists():
    print(f'Exists {RESULT_DIR_PATH}')
else:
    #RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_DIR_PATH}')

print('-------------------------------------------')
if IMEL_DIR_PATH.exists():
    print(f'Exists {IMEL_DIR_PATH}')
else:
    print(f'No exist {IMEL_DIR_PATH}')

print('-------------------------------------------')
if RESULT_WAV_DIR_PATH.exists():
    print(f'Exists {RESULT_WAV_DIR_PATH}')
else:
    RESULT_WAV_DIR_PATH.mkdir(parents=True)
    print(f'Make {RESULT_WAV_DIR_PATH}')

-------------------------------------------
Exists configs/test_dataset.json
loaded configs/test_dataset.json
-------------------------------------------
Exists result4eval/infer4colb/gradfreqktts/cpu/e500_n50
-------------------------------------------
Exists result4eval/infer4colb/gradfreqktts/cpu/e500_n50/mel
-------------------------------------------
Make result4eval/infer4colb/gradfreqktts/cpu/e500_n50/wav_LJ_V1


In [7]:

# Setup HiFi-GAN ====================
"""
from hifigan import models, env

with open(HiFiGAN_CONFIG) as f:
    hifigan_hparams = env.AttrDict(json.load(f))
"""
#hifigan
# setting file paths
# from https://github.com/huawei-noah/Speech-Backbones/tree/main/Grad-TTS/hifi-gan
# https://drive.google.com/drive/folders/1-eEYTB5Av9jNql0WGBlRoi-WH2J7bp5Y?usp=sharing
if hifigan_versions == 'LJ_V1':
    HiFiGAN_CONFIG = f'./hifigan/official_pretrained/LJ_V1/config.json'
    HiFiGAN_ckpt = './hifigan/official_pretrained/LJ_V1/generator_v1'
elif hifigan_versions == 'LJ_V2':
    HiFiGAN_CONFIG = './hifigan/official_pretrained/LJ_V2/config.json'
    HiFiGAN_ckpt = './hifigan/official_pretrained/LJ_V2/generator_v2'
else:
    print('Dont supported.')

from hifigan import models, env

with open(HiFiGAN_CONFIG) as f:
    hifigan_hparams = env.AttrDict(json.load(f))
print(f'hifigan_versions:{hifigan_versions}')
print(f'hifigan_samplerate:{hifigan_hparams.sampling_rate}')
# generator ===================
print("[seq] loading HiFiGAN")
vocoder = models.Generator(hifigan_hparams)

vocoder.load_state_dict(torch.load(
    HiFiGAN_ckpt, map_location=device)['generator'])
vocoder = vocoder.eval().to(device)
vocoder.remove_weight_norm()

# Setup utmos ===================
print("[seq]loading UTMOS")
predictor_utmos = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)



hifigan_versions:LJ_V1
hifigan_samplerate:22050
[seq] loading HiFiGAN


/work/sora-sa/aoi/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Removing weight norm...
[seq]loading UTMOS


Using cache found in /work/sora-sa/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0


In [8]:
infer_data_num: int = 101 #len(test_ds_list) is 200
print(f'infer_data_num: {infer_data_num}')
print(f'RESULT_DIR_PATH: {RESULT_DIR_PATH}')
print(f'IMEL_DIR_PATH: {IMEL_DIR_PATH}')
print(f'RESULT_WAV_DIR_PATH: {RESULT_WAV_DIR_PATH}')
print(f'RESULT_JSON_PATH: {RESULT_JSON_PATH}')

infer_data_num: 101
RESULT_DIR_PATH: result4eval/infer4colb/gradfreqktts/cpu/e500_n50
IMEL_DIR_PATH: result4eval/infer4colb/gradfreqktts/cpu/e500_n50/mel
RESULT_WAV_DIR_PATH: result4eval/infer4colb/gradfreqktts/cpu/e500_n50/wav_LJ_V1
RESULT_JSON_PATH: result4eval/infer4colb/gradfreqktts/cpu/e500_n50/eval4mid_LJ_V1.json


## [seq] infer

In [9]:
eval_list = []


for i in tqdm(range(infer_data_num)):
    test_ds_filename = test_ds_list[i]['name']
    mel_npy_path = IMEL_DIR_PATH / f"{test_ds_filename}.npy"
    synth_wav_path = RESULT_WAV_DIR_PATH / f"{test_ds_filename}.wav"
    print(f'test_ds_index_{i}: {test_ds_filename}')
    # [seq]mel2wav =========================================================
    print('[seq]mel2wav')
    x = np.load(mel_npy_path) # [1, n_mel, n_frame]
    x2audio = torch.FloatTensor(x).to(device)
    x2audio = x2audio.squeeze().unsqueeze(0)
    # x2audio is [1, n_mels, n_frames]
    assert x2audio.shape[0] == 1
    start_time = time.perf_counter()
    with torch.no_grad():
        # vocoder.forward(x).cpu() is torch.Size([1, 1, 167168])
        audio = (vocoder.forward(x2audio).cpu().squeeze().clamp(-1,1).numpy() * 32768).astype(np.int16)
    end_time = time.perf_counter()
    dt_wav = end_time - start_time
    replay_time = audio.shape[-1] / sample_rate
    rtf4wav = dt_wav / replay_time
    #dt4mel = dt * 22050 / ( mel_prediction.shape[-1] * 256)
    #print(f'{model_name} dt: {dt}')
    #print(f'{model_name} RTF: {dt4mel}')
    write(
        synth_wav_path,
        sample_rate,
        audio)

    # [seq]wav2utmos =========================================================
    print('[seq]wav2utmos')
    #iwav_path = RESULT_WAV_DIR_PATH / f"{filename}.wav"
    #wav, samplerate = torchaudio.load(iwav_path)
    wav, samplerate = torchaudio.load(synth_wav_path)
    score_utmos = predictor_utmos(wav, sample_rate)
    score_utmos_float = score_utmos.item()
    print(f'utmos: {score_utmos_float}')

    eval_dict = {
        'name': str(test_ds_filename),
        'wav_path': str(synth_wav_path),
        'dt4mwav': dt_wav,
        'RTF4mwav': rtf4wav,
        'utmos': score_utmos_float
    }
    eval_list.append(eval_dict)
    

  0%|                                                                                                                                            | 0/101 [00:00<?, ?it/s]

test_ds_index_0: LJ045-0049
[seq]mel2wav
[seq]wav2utmos


  1%|█▎                                                                                                                                  | 1/101 [00:04<08:11,  4.91s/it]

utmos: 4.116062164306641
test_ds_index_1: LJ017-0027
[seq]mel2wav


  2%|██▌                                                                                                                                 | 2/101 [00:06<04:56,  3.00s/it]

...

utmos: 4.139237880706787
test_ds_index_99: LJ029-0081
[seq]mel2wav
[seq]wav2utmos


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 100/101 [05:54<00:05,  5.01s/it]

utmos: 3.969770669937134
test_ds_index_100: LJ017-0230
[seq]mel2wav
[seq]wav2utmos


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [05:59<00:00,  3.56s/it]

utmos: 4.005331039428711


In [13]:
#RESULT_JSON_PATH = RESULT_DIR_PATH / 'eval4mid.json'
if RESULT_JSON_PATH.exists() == False:
    with open(RESULT_JSON_PATH, 'w') as f:
        for entry in eval_list:
            f.write(json.dumps(entry) + '\n')
    print(f'Make {RESULT_JSON_PATH}')
else:
    print(f'Already Exists {RESULT_JSON_PATH}')

Already Exists result4eval/infer4colb/gradfreqktts/cpu/e500_n50/eval4mid_LJ_V1.json


In [16]:
#a = toybox.load_json('result4eval/run_tfkfulmask_k3/gradtfkfultts/cpu/e500/eval4mid.json')
#a = toybox.load_json('result4eval/infer4colb/gradtts/cpu/e500_n50/eval4mid_LJ_V1.json')
#a = toybox.load_json('result4eval/infer4colb/gradseptts/cpu/e500_n50/eval4mid_LJ_V1.json')
#a = toybox.load_json('result4eval/infer4colb/gradtfktts/cpu/e500_n50/eval4mid_LJ_V1.json')
#a = toybox.load_json('result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/eval4mid_LJ_V1.json')
#a = toybox.load_json('result4eval/infer4colb/gradtfkful_mask/cpu/e500_n50/eval4mid_LJ_V1.json')
#a = toybox.load_json('result4eval/infer4colb/gradtfkful_plus/cpu/e500_n50/eval4mid_LJ_V1.json')
#a = toybox.load_json('result4eval/infer4colb/gradtimektts/cpu/e500_n50/eval4mid_LJ_V1.json')
a = toybox.load_json('result4eval/infer4colb/gradfreqktts/cpu/e500_n50/eval4mid_LJ_V1.json')

Exist result4eval/infer4colb/gradfreqktts/cpu/e500_n50/eval4mid_LJ_V1.json


In [17]:
dt_list = [a[n]['dt4mwav'] for n in range(len(a))]
RTF4mel_list = [a[n]['RTF4mwav'] for n in range(len(a))]
utmos_list = [a[n]['utmos'] for n in range(len(a))]

dt_nparr = np.array(dt_list[1:101])
RTF4mel_nparr = np.array(RTF4mel_list[1:101])
utmos_nparr = np.array(utmos_list[1:101])
print(len(dt_nparr))

significant_digits = 8

# for culc difference time to infer text2mel
dt_mean = toybox.round_significant_digits(np.mean(dt_nparr), significant_digits=significant_digits)
dt_var = toybox.round_significant_digits(np.var(dt_nparr), significant_digits=significant_digits)
dt_std = toybox.round_significant_digits(np.std(dt_nparr), significant_digits=significant_digits)
print(f'dt ---------------------------')
print(f'dt mean: {dt_mean}')
print(f'dt var: {dt_var}')
print(f'dt std: {dt_std}')

# for culc RTF4mel to infer text2mel
RTF4mel_mean = toybox.round_significant_digits(np.mean(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_var = toybox.round_significant_digits(np.var(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_std = toybox.round_significant_digits(np.std(RTF4mel_nparr), significant_digits=significant_digits)
print(f'RTF ---------------------------')
print(f'RTF mean: {RTF4mel_mean}')
print(f'RTF var: {RTF4mel_var}')
print(f'RTF std: {RTF4mel_std}')

# for culc utmos to infer 
print(f'utmos ---------------------------')
utmos_mean = toybox.round_significant_digits(np.mean(utmos_nparr), significant_digits=significant_digits)
utmos_var = toybox.round_significant_digits(np.var(utmos_nparr), significant_digits=significant_digits)
utmos_std = toybox.round_significant_digits(np.std(utmos_nparr), significant_digits=significant_digits)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

100
dt ---------------------------
dt mean: 3.1302962
dt var: 1.7720768
dt std: 1.3311937
RTF ---------------------------
RTF mean: 0.48584148
RTF var: 0.011913627
RTF std: 0.10914956
utmos ---------------------------
utmos mean: 4.0460148
utmos var: 0.08215180400000001
utmos std: 0.28662136


## for test

In [30]:
# for text2mel
print('test_ds_path-----------------------------------------')
if test_ds_path.exists():
    print(f'Exists {str(test_ds_path)}')
    with open(config['test_datalist_path']) as j:
        test_ds_list = json.load(j)
    print(f'loaded {test_ds_path}')
else:
    print(f'No exist {test_ds_path}')

# test plot
x = np.load(IMEL_DIR_PATH / f"{test_ds_list[0]['name']}.npy") # [1, n_mel, n_frame]
print(IMEL_DIR_PATH / f"{test_ds_list[0]['name']}.npy")
x4plot = torch.FloatTensor(x).to(device)
x4plot = x4plot.squeeze().cpu()
print(x4plot.shape)
fig_meltmp = toybox.plot_mel([x4plot], ['mel2wavtmp'])

test_ds_path-----------------------------------------
Exists configs/test_dataset.json
loaded configs/test_dataset.json
result4eval/infer4colb/gradtts/cpu/e500_n50/mel/LJ045-0049.npy
torch.Size([80, 667])


In [33]:
x2audio = torch.FloatTensor(x).to(device)
x2audio = x2audio.squeeze().unsqueeze(0)
print(f'x2audio.shape: {x2audio.shape}')
# x2audio is [1, n_mels, n_frames]
assert x2audio.shape[0] == 1
with torch.no_grad():
    # vocoder.forward(x).cpu() is torch.Size([1, 1, 167168])
    audio = (vocoder.forward(x2audio).cpu().squeeze().clamp(-1,1).numpy() * 32768).astype(np.int16)

ipd.display(ipd.Audio(audio, rate=22050))

x2audio.shape: torch.Size([1, 80, 667])


In [37]:
audio.shape[-1]/22050

7.743854875283446

In [32]:
#a = audio.detach().clone()
a = torch.from_numpy(audio)
a.shape
audio.shape[-1]

torch.Size([165376])

In [33]:
#result4eval/infer4PBL/gradseptts/cpu/wav/LJ045-0049.wav
#torch.Size([1, 165376])
a = a.unsqueeze(0)
a.shape

torch.Size([1, 165376])

In [34]:
# Convert to float32 if needed
if not torch.is_floating_point(a):
    a = a.to(torch.float32) / 32768.0 

In [35]:
print(sample_rate)
#score_utmos = predictor_utmos(wav, samplerate)
score_utmos = predictor_utmos(a, sample_rate)
score_utmos_float = score_utmos.item()
print(f'utmos: {score_utmos_float}')

22050
utmos: 4.314143657684326
